# Notebook 2: Expériences sur les modèles

**Auteurs:**  

Akram Farihi, Sami Abloui, Amalya Mourih 

**Objectifs de notebook:**

Dans ce notebook, nous réalisons l'entraînement sur les modèles choisis avec accomagnement de MLFlows et le reste. Nous avons le code preprocessing.py qui permettra de faire le prétraiment des données. et nous allons effectue nos expéricences



In [2]:
# les lib importé 
from sys import path

path.append('..')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from src.data.preprocessing  import preprocess_data
from src.data.data_loader import load_data
from src.utils.mlflow_run_manager import save_run

import mlflow

print("Les bibliothèques sont importées avec succès.")

Les bibliothèques sont importées avec succès.


## 1. Chargement et séléction des données

Nous avons créé un code spécifique qui permettra d'effectuer le prétraitement des données avant avant l'entraînement des modèles.

In [3]:
df_raw = load_data()
print("Données chargées avec succès.")

print("\n Prétraitement des données...")
print("\n -------------------------------------------------------------------------")
df = preprocess_data(df_raw)
print("Prétraitement des données terminé.")
print("Aperçu des données prétraitées:")
print(df.head())

Données chargées avec succès.

 Prétraitement des données...

 -------------------------------------------------------------------------
   ID  Age  Gender  Country  Coffee_Intake  Caffeine_mg  Sleep_Hours  \
0   1   40    Male  Germany            3.5        328.1          7.5   
1   2   33    Male  Germany            1.0         94.1          6.2   
2   3   42    Male   Brazil            5.3        503.7          5.9   
3   4   53    Male  Germany            2.6        249.2          7.3   
4   5   32  Female    Spain            3.1        298.0          5.3   

  Sleep_Quality   BMI  Heart_Rate Stress_Level  Physical_Activity_Hours  \
0          Good  24.9          78          Low                     14.5   
1          Good  20.0          67          Low                     11.0   
2          Fair  22.7          59       Medium                     11.2   
3          Good  24.7          71          Low                      6.6   
4          Fair  24.1          76       Medium         

Pour notre projet, nous n'analysons pas directement le niveau de stress et les problèmes de santé, mais nous cherchons à déterminer l'impact (ou l'issue) principal du stress et des problèmes de santé en examinant la qualité du sommeil, qui sera notre variable cible (ou variable à prédire).
Nous allons faire un train-test split en utilisant les données comme variables d’entraînement et la colonne de qualité du sommeil comme variable cible.

In [80]:

X = df.drop(columns=['Sleep_Quality', 'Sleep_Hours'])
y = df['Sleep_Quality']

# appliquer une répartition stratifiée car nos données sont déséquilibrées

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

## 2. Configuration des modèles 
Nous allons ensuite construire un pipeline afin d’automatiser la normalisation des données, l’entraînement des modèles et la définition des bonnes grilles de paramètres. Nous avons choisi d’utiliser des modèles de classification plutôt que des modèles de régression, car notre objectif est de prédire une classe. Trois approches seront donc évaluées :

- SVM, un classifieur performant et adapté aux jeux de données de taille moyenne.

- RandomForest, un ensemble d'arbres de décision robuste, efficace sur données tabulaires et permettant d'estimer l'importance des variables.

- MLP, un réseau de neurones adapté aux données tabulaires, contrairement aux CNN destinés aux images et aux RNN conçus pour les données séquentielles.

In [81]:

pipelines = {
    "rf": Pipeline([
        ("scaler", MinMaxScaler()),
        ("model", RandomForestClassifier())
    ]),
    "svm": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVC(probability=True))
    ]),
    "mlp": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPClassifier(max_iter=300))
    ])
}

param_grids = {
    "rf": {
        "model__n_estimators": [500],      
        "model__max_depth": [5],       
        "model__min_samples_split": [7],      
        "model__min_samples_leaf": [12],       
        "model__bootstrap": [True]            
    },

    "svm":[ 
        
         # RBF kernel
        # {
        #     "model__C": [20, 50, 100],
        #     "model__kernel": ["rbf"],
        #     "model__gamma": [0.05, 0.5, 5, 25]
        # },
        # Linear kernel
        {
            "model__C": [1],
            "model__kernel": ["linear"]
        }
        # Poly kernel
        # {
        #     "model__C": [0,.8, 0.9, 1, 1.1, 1.2],
        #     "model__kernel": ["poly"],
        #     "model__degree": [2, 3],
        #     "model__gamma": [0.01, 0.1, 1]
        # }
    ],

    "mlp": {
        "model__hidden_layer_sizes": [(35,)],
        "model__activation": ["relu", "tanh", "logistic"],
        "model__solver": ["adam"],
        "model__alpha": [0.004],
        "model__learning_rate": ["adaptive"]
    }
}

## 3. Entraînements et recherche de meilleures paramètres

Nous allons ensuite exécuter une boucle GridSearchCV, combinant recherche par grille et validation croisée, afin d’optimiser les hyperparamètres de chaque modèle et déterminer le meilleur pour notre problème


In [82]:
best_models = {}
best_scores = {}

experiment_name = "07_12_comparing_models_without_sleep_hours"

mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment(experiment_name)

for name in pipelines:
    print("Recherche modèle :", name)

    with mlflow.start_run(run_name=name) as run:

        scoring = {
            "accuracy": "accuracy",
            "recall_weighted": "recall_weighted",
            "precision_weighted": "precision_weighted",
            "f1_weighted": "f1_weighted"
        }
   
        grid = GridSearchCV(
            estimator=pipelines[name],
            param_grid=param_grids[name],
            cv=3,
            scoring=scoring,
            refit="f1_weighted",
            n_jobs=-1
        )
        
        grid.fit(X_train, y_train)
        
        best_models[name] = grid.best_estimator_
        best_scores[name] = grid.best_score_
                
        # Log metrics
        mlflow.log_metric("best_cv_score", grid.best_score_)
        for metric_name, metric_value in scoring.items():
            if metric_name in grid.cv_results_:
                mlflow.log_metric(f"mean_test_{metric_name}", grid.cv_results_[f'mean_test_{metric_name}'][grid.best_index_])
        
        mlflow.sklearn.log_model(grid.best_estimator_, artifact_path="model")

        # Log parameters
        mlflow.log_params(grid.best_params_)

        print("Meilleurs paramètres :", grid.best_params_)
        print("Score CV :", grid.best_score_)
        print()

        save_run(experiment_name, run.info.run_id, grid.best_score_, name)




Recherche modèle : rf


2025/12/08 00:26:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/08 00:26:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'model__bootstrap': True, 'model__max_depth': 5, 'model__min_samples_leaf': 12, 'model__min_samples_split': 7, 'model__n_estimators': 500}
Score CV : 0.8043695611046325

Le fichier a été mis à jour.
Recherche modèle : svm


2025/12/08 00:26:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/08 00:26:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'model__C': 1, 'model__kernel': 'linear'}
Score CV : 0.8043695611046325

Le fichier a été mis à jour.
Recherche modèle : mlp


c:\Users\amaly\OneDrive\Bureau\Amu M2\Apprentissage automatique pour DS\pd\Processus_data\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
2025/12/08 00:26:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/08 00:26:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'model__activation': 'relu', 'model__alpha': 0.004, 'model__hidden_layer_sizes': (35,), 'model__learning_rate': 'adaptive', 'model__solver': 'adam'}
Score CV : 0.8077276998558575

Le fichier a été mis à jour.


In [83]:
best_model_name = max(best_scores, key=best_scores.get)
best_model = best_models[best_model_name]

print("Meilleur modèle :", best_model_name)
print(hasattr(best_model, "predict_proba"))


Meilleur modèle : mlp
True


In [84]:
print(best_scores)

{'rf': np.float64(0.8043695611046325), 'svm': np.float64(0.8043695611046325), 'mlp': np.float64(0.8077276998558575)}


## 4. Résultats et sélection du meilleur modèle

Après avoir entraîné les trois modèles (Random Forest, SVM et MLP) avec GridSearchCV et validation croisée, nous avons identifié le meilleur modèle en comparant leurs scores F1 pondérés. Les résultats montrent que le **SVM avec noyau linéaire** obtient les meilleures performances avec un score de **99.26%**, suivi de près par Random Forest (99.18%) et MLP (99.16%). Tous les modèles démontrent d'excellentes capacités de prédiction de la qualité du sommeil, mais le SVM est retenu comme modèle optimal pour la suite du projet. Les hyperparamètres optimaux et les métriques de performance ont été automatiquement enregistrés dans MLflow pour assurer la traçabilité et la reproductibilité des expériences.